In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.optimizers import Adam
from keras.layers import Dense, Input, TimeDistributed, Flatten, Dropout
from keras.models import Sequential, Model
from keras.layers import Conv2D, BatchNormalization, LeakyReLU, Reshape, Conv2DTranspose
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from tensorflow.keras.layers import Conv3D,Conv3DTranspose, BatchNormalization, LeakyReLU


In [2]:
def make_3D_discriminator_model(input_shape=(8,64,64,3)):
    model = tf.keras.Sequential()

    model.add(Conv3D(128, (3, 3,3), strides=(1,1,1), padding='same', input_shape=input_shape))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3D(128, (4, 4, 4), strides=(2,2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3D(128, (4, 4,4), strides=(2,2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3D(128, (4, 4,4), strides=(2,2,2), padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3D(128, (4, 4,4), strides=(1,2, 2), padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation="sigmoid"))
    opt = Adam(learning_rate=0.0001, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model
test_d_model= make_3D_discriminator_model()
print(test_d_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 8, 64, 64, 128)    10496     
                                                                 
 batch_normalization (BatchN  (None, 8, 64, 64, 128)   512       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 8, 64, 64, 128)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 4, 32, 32, 128)    1048704   
                                                                 
 batch_normalization_1 (Batc  (None, 4, 32, 32, 128)   512       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 4, 32, 32, 128)    0

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
def make_3D_generator_model():
    codings_size = 100
    model = Sequential()
    model.add(Dense(16*16*128, use_bias=False, input_shape=[codings_size]))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Reshape((1,16, 16, 128)))
    assert model.output_shape == (None,1, 16, 16, 128)

    model.add(Conv3DTranspose(128, (4, 4,4), strides=(1,2, 2), padding='same'))
    assert model.output_shape == (None, 1,32, 32, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3DTranspose(128, (5, 5,5), strides=(2,1, 1), padding='same'))
    assert model.output_shape == (None,2,32, 32, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3DTranspose(128, (5, 5,5), strides=(2,1,1), padding='same'))
    assert model.output_shape == (None,4, 32, 32, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3DTranspose(128, (5, 5,5), strides=(2,2,2), padding='same'))
    assert model.output_shape == (None,8, 64, 64, 128)
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU(0.1))

    model.add(Conv3DTranspose(3, (5, 5,5), strides=(1, 1,1), padding='same', activation="tanh"))
    assert model.output_shape == (None,8, 64, 64, 3)

    return model
test_g_model= make_3D_generator_model()
print(test_g_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 32768)             3276800   
                                                                 
 batch_normalization_5 (Batc  (None, 32768)            131072    
 hNormalization)                                                 
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 32768)             0         
                                                                 
 reshape (Reshape)           (None, 1, 16, 16, 128)    0         
                                                                 
 conv3d_transpose (Conv3DTra  (None, 1, 32, 32, 128)   1048704   
 nspose)                                                         
                                                                 
 batch_normalization_6 (Batc  (None, 1, 32, 32, 128)  

In [4]:
def define_3D_DCGAN(generator, discriminator, latent_dim):
	discriminator.trainable = False  #Discriminator is trained separately. So set to not trainable.
	# connect generator and discriminator
	model = Sequential()
	model.add(generator)
	model.add(discriminator)
	# compile model
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model
dcgan_3D_model_structure = define_3D_DCGAN(discriminator=test_d_model, generator=test_g_model, latent_dim=100)
print(dcgan_3D_model_structure.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 8, 64, 64, 3)      10651011  
                                                                 
 sequential (Sequential)     (None, 1)                 4209921   
                                                                 
Total params: 14,860,932
Trainable params: 10,584,451
Non-trainable params: 4,276,481
_________________________________________________________________
None
